# Hull White Analysis

In [1]:
import random_number as rn
from matplotlib import pyplot as plt
import short_rate_model as srm
import numpy as np
import scipy.stats as stats
import math

### Hull White model:

The Hull White short rate model is defined as:
\begin{align}
dr_t = k \big( \theta(t) - r_t \big) \, dt + \sigma(t) \, dW_t
\end{align}
i.e. it is identical to the Vasicek model, but the long term mean $\theta(t)$ and short rate vol $\sigma(t)$ are now functions of time. 

We can solve the HW SDE with the same trick we used for solving the Vasicek SDE:
\begin{align}
d\big(\exp(kt) \, r_t \big) = \exp(kt) \, \tilde{\theta}(t) \, dt + \exp(kt) \, \sigma(t) \, dW_t.
\end{align}
where $\tilde{\theta}(t) \equiv \theta(t) \ k$. Integrating both sides from $t$ to $s$ and rearranging terms, we get
\begin{align}
\boxed{r_s = \exp(-k(s-t)) \, r_t + \int_t^s \exp(-k(s-u)) \, \tilde{\theta}(u) \, du + \int_t^s \exp(-k(s-u)) \, \sigma(u) \, dW_u}
\end{align}

The HW model implied ZCB price derivation follows almost exactly as that of the Vasicek model. Let $X_t \equiv \int_t^T r_u \, du$, then the ZCB price at time $t$ is given as:
\begin{align}
B(t, T) &= \mathbb{E}^{\mathbb{Q}}_t \big[\exp \big( -X_t \big) \big] \\
        &= \exp \bigg( -\mathbb{E}^{\mathbb{Q}}_t[X_t] + \frac{1}{2} \mathbb{V}ar[X_t] \bigg)
\end{align}
with the mean and variance of $X_t \equiv \int_t^T r_u \, du$ given as
\begin{align}
\mathbb{E} [X_t] &= r_t \, G(t, T) + \int_t^T \tilde{\theta}(u) \, G(u, T) \, du \\
\mathbb{V}ar [X_T] &= \int_t^T \sigma^2(u) \, G^2(u, T \,|\, k) \, du
\end{align}
where the $G(0, T \,|\, k)$ is the same as the one used in the Vasicek analysis:
\begin{align}
G(t,s \,|\, k) &\equiv \int_t^s \exp(-k(s-u)) \, du \\
                &= \frac{1 - \exp(-k(s-t))}{k}
\end{align}
Putting everything together, the HW implied ZCB is given as
\begin{align}
\boxed{B(t,T) = \exp \bigg( -r_t \, G(t,T) - \int_t^T \tilde{\theta}(u) \, G(u,T) \, du + \frac{1}{2} \int_t^T \sigma^2(u) \, G^2(u,T) \, du  \bigg)}
\end{align}

The goal here is to extract $\tilde{\theta}(u)$ and calibrate it such that it fits the initial discount curve $T \mapsto B(0,T)$ exactly. We shall proceed by assuming a known (calibrated) short rate vol $\sigma$.

$\tilde{\theta}(u)$ can be calibrated numerically using the day-0 discount curve, $T \mapsto B(0,T)$:

\begin{align}
B(0,T) = \exp \bigg( -r_0 \, G(0,T \,|\, k) - \int_0^T \tilde{\theta}(u) \, G(u,T \,|\, k) \, du + \frac{1}{2} \int_0^T \sigma^2(u) \, G^2(u,T \,|\, k) \, du  \bigg)
\end{align}

\begin{align}
\implies \int_0^T \tilde{\theta}(u) \, G(u,T \,|\, k) \, du = -\ln B(0,T) + \frac{1}{2} \int_0^T \sigma^2(u) \, G^2(u,T \,|\, k) \, du - r_0 \, G(0,T \,|\, k) 
\end{align}

If we assume piecewise constant $\tilde{\theta}(u)$, then:
\begin{align}
\sum_{i=1}^{N-1} \bigg( \tilde{\theta}(t_{i}) \int_{t_{i}}^{t_{i+1}} G(u,T \,|\, k) \, du\bigg) = -\ln B(0,T) + \frac{1}{2} \int_0^T \sigma^2(u) \, G^2(u,T \,|\, k) \, du - r_0 \, G(0,T \,|\, k) 
\end{align}
where $t_0=0$ and $t_N=T$. The function $\tilde{\theta}(u)$ can now be bootstrapped using the day-0 discount curve, $T \mapsto B(0,T)$.

We are also interested in calibrating $\tilde{\theta}(u)$ analytically. More directly, we aim to express the integral $\int_t^T \tilde{\theta}(u) \, G(u,T) \, du$ as a function of day-0 market quantities. To start, we have

\begin{align}
\ln B(0,T) &= -r_0 \, G(0,T) - \int_0^T \tilde{\theta}(u) \, G(u,T) \, du + \frac{1}{2} \int_0^T \sigma^2(u) \, G^2(u,T) \, du   \\
\ln B(0,t) &= -r_0 \, G(0,t) - \underbrace{\int_0^t \tilde{\theta}(u) \, G(u,t) \, du}_{\color{red}{\text{A}}} + \underbrace{\frac{1}{2} \int_0^t \sigma^2(u) \, G^2(u,t) \, du}_{\color{blue}{\text{B}}}
\end{align}

Taking the difference of the two equations will give us the term we wanted: $\int_t^T \tilde{\theta}(u) \, G(u,T) \, du$. Specifically focusing on the second terms ($\color{red}{\text{A}}$), we have:
\begin{align}
&\int_0^T \tilde{\theta}(u) \, G(u,T) \, du - \int_0^t \tilde{\theta}(u) \, G(u,t) \, du \\
&\;\;\;\;\;\;\; = 
\int_t^T \tilde{\theta}(u) \, G(u,T) \, du + \int_0^t \tilde{\theta}(u) \, \big( \underbrace{G(u,T ) - G(u,t)}_{\partial_t G(u,t) \, G(t,T)} \big) \, du \\
&\;\;\;\;\;\;\; =
\underbrace{\int_t^T \tilde{\theta}(u) \, G(u,T) \, du}_{\text{what we want}} + \underbrace{G(t,T) \int_0^t \tilde{\theta}(u) \, \partial_t G(u,t) \, du}_{\text{extra term}}
\end{align}

The extra term $G(t,T) \int_0^t \tilde{\theta}(u) \, \partial_t G(u,t) \, du$ can be eliminated by noting that:
\begin{align}
f(0,t) &\equiv -\partial_t \ln B(0,t) \\
        &= r_0 \, \partial_t G(0,t) + \int_0^t \tilde{\theta}(u) \, \partial_t G(u,t) \, du - \int_0^t \sigma^2  G(u, t) \, \partial_t G(u,t) \, du
\end{align}
\begin{align}
\implies 
G(t,T) \int_0^t \tilde{\theta}(u) \, \partial_t G(u,t) \, du
= f(0,t) G(t,T) - r_0 \underbrace{G(t,T) \partial_t G(0,t)}_{G(0,T) - G(0,t)} + G(t,T) \int_0^t \sigma^2  G(u, t) \, \partial_t G(u,t) \, du
\end{align}

Focusing on the third terms ($\color{blue}{\text{B}}$), we have:
\begin{align}
&\frac{1}{2} \int_0^T \sigma^2(u) \, G^2(u,T) \, du - \frac{1}{2} \int_0^t \sigma^2(u) \, G^2(u,t) \, du \\
&\;\;\;\;\;\;\; = 
\frac{1}{2} \int_t^T \sigma^2(u) \, G^2(u,T) \, du + \frac{1}{2} \int_0^t \sigma^2(u) \big( G^2(u,T) - G^2(u,t) \big) \, du \\
&\;\;\;\;\;\;\; = 
\frac{1}{2} \int_t^T \sigma^2(u) \, G^2(u,T) \, du + \frac{1}{2} \int_0^t \sigma^2(u) \partial_t G(u,t) G(t,T) \bigg( G(u,T) + G(u,t) \bigg) \, du \\
&\;\;\;\;\;\;\; = 
\frac{1}{2} \int_t^T \sigma^2(u) \, G^2(u,T) \, du + G(t,T) \int_0^t \sigma^2(u) \, G(u,t) \, \partial_t G(u,t) \, du + \frac{1}{2} G^2(t,T) \int_0^t \sigma^2(u) \, \partial_t G^2(u,t) \, du
\end{align}


Putting everything together, we have $\ln B(0,T) - \ln B(0,t)$ given as:
\begin{align}
\ln \frac{B(0,T)}{B(0,t)} 
&= - \underbrace{\int_t^T \tilde{\theta}(u) G(u,T) \, du}_{\text{what we want (from ($\color{red}{\text{A}}$))}} \\
& \;\;\;\;\;\;\; \underbrace{-f(0,t) G(t,T) + r_0 \big(G(0,T) - G(0,t)\big) - G(t,T) \int_0^t \sigma^2  G(u, t) \, \partial_t G(u,t) \, du}_{\text{eliminating extra term (from ($\color{red}{\text{A}}$))}} \\
& \;\;\;\;\;\;\; + \underbrace{\frac{1}{2} \int_t^T \sigma^2(u) \, G^2(u,T) \, du + G(t,T) \int_0^t \sigma^2(u) \, G(u,t) \, \partial_t G(u,t) \, du + \frac{1}{2} G^2(t,T) \int_0^t \sigma^2(u) \, \partial_t G^2(u,t) \, du}_{\text{from ($\color{blue}{\text{B}}$)}} \\
&= -f(0,t) G(t,T) 
+ r_0 \big(G(0,T) - G(0,t)\big)
-\underbrace{\int_t^T \tilde{\theta}(u) G(u,T) \, du}_{\text{what we want}} \\
& \;\;\;\;\;\; + \frac{1}{2} \int_t^T \sigma^2(u) \, G^2(u,T) \, du
+ \frac{1}{2} G^2(t,T) \int_0^t \sigma^2(u) \, \partial_t G^2(u,t) \, du
\end{align}
Rearranging, we now have an analytical expression of the integral $\int_t^T \tilde{\theta}(u) G(u,T) \, du$ in terms of day-0 market quantities:
\begin{align}
\implies
\underbrace{\int_t^T \tilde{\theta}(u) G(u,T) \, du}_{\text{what we want}}
&= -\ln \frac{B(0,T)}{B(0,t)}
-f(0,t) G(t,T) 
+ r_0 \big(G(0,T) - G(0,t)\big) \\
& \;\;\;\;\;\;\; + \frac{1}{2} \int_t^T \sigma^2(u) \, G^2(u,T) \, du
+ \frac{1}{2} G^2(t,T) \int_0^t \sigma^2(u) \, \partial_t G^2(u,t) \, du 
\end{align}

Substituting the integral back into our HW ZCB equation:
\begin{align}
B(t,T) = \exp \bigg( -r_t \, G(t,T) - \underbrace{\int_t^T \tilde{\theta}(u) \, G(u,T) \, du}_{\text{substitude here}} + \frac{1}{2} \int_t^T \sigma^2(u) \, G^2(u,T) \, du  \bigg)
\end{align}
We now have
\begin{align}
B(t,T) = 
\frac{B(0,T)}{B(0,t)} \, 
\exp \bigg( -r_t \, G(t,T) + f(0,t) \, G(t,T) - \underbrace{r_0 \big(G(0,T) - G(0,t)\big)}_{\text{where?!}} - \frac{1}{2} G^2(t,T) \int_0^t \sigma^2(u) \, \partial_t G^2(u,t) \, du \bigg)
\end{align}
Tidying up, we have
\begin{align}
\boxed{B(t,T) = 
\frac{B(0,T)}{B(0,t)} \, 
\exp \bigg( -\underbrace{\big(r_t - f(0,t) \big)}_{x_t} \, G(t,T) - \frac{1}{2} G^2(t,T) \underbrace{\int_0^t \sigma^2(u) \, \partial_t G^2(u,t) \, du}_{y_t} \,\bigg)}
\end{align}
where $\partial_t G^2(u,t) = \exp \big( -2k (t-u) \big)$.